In [8]:
import shutil
import multiprocessing
import ast
import subprocess
import re
import os
import shlex
from functools import partial
class log_wrapper:
    def __init__(self, cmdline, tag):
        self.cmdline = cmdline
        self.tag = tag

    def __enter__(self):
        print(f"Running: {self.cmdline}")

    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type:
            print(f"Error occurred in {self.tag}: {exc_value}")
        else:
            print(f"{self.tag} completed successfully.")
class SensitivityAnalysis():
    def __init__(self):
        self.file_setup = "setup/YKD-burned_setup.dat"
        self.file_driver = "setup/YKD-burned_driver.dat"
        self.file_data = "data/YKD-burned.dat"
        self.number = 0
        self.target_string = ""
        self.file_list = []
        self.target_values = []
        self.dictionaries = []
        self.list_of_modifies_files = []

    @staticmethod
    def write_dictionary_to_file(dictionary, filename):
        with open(filename, 'w') as file:
            for key, value in dictionary.items():
                file.write(f"{key}\t{value}\n")
            file.write("#\n")

    def create_files(self):
        self.number = len(self.list_of_modifies_files)
        for i, (setup_file, driver_file) in enumerate(self.list_of_modifies_files):
            setup_filename = f"setup/YKD{i}-burned_setup.dat"
            driver_filename = f"setup/YKD{i}-burned_driver.dat"

            self.write_dictionary_to_file(setup_file, setup_filename)
            self.write_dictionary_to_file(driver_file, driver_filename)

        return self.file_list
    def _copy_files(self,args):
        source_file,destination_file = args
        with open(source_file, "r") as setup_file, open(destination_file, "w") as new_setup_file:
            shutil.copyfileobj(setup_file, new_setup_file)
    @staticmethod
    def process_file(file_path):
        with open(file_path, 'r') as file:
            lines = file.readlines()

        dictionary = {}
        for line in lines:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            elif line == "end":
                # Include "end" as a key without any associated value
                dictionary["end"] = None
                break  # Stop processing further lines after encountering "end"
            else:
                # Using regular expression to extract key and value parts
                match = re.match(r"^(\S+)\s+(.*)$", line)
                if match:
                    key = match.group(1)
                    value = match.group(2).strip()
                    dictionary[key] = value

        return dictionary

   






    def create_data_file(self):
        try:
            if not os.path.exists("data"):
                os.makedirs("data")

            # Prepare arguments for multiprocessing
            file_args = [(self.file_data, f"data/YKD{i}-burned.dat") for i in range(len(self.list_of_modifies_files))]

            # Create a multiprocessing pool with the number of CPU cores available
            num_processes = multiprocessing.cpu_count()

            with multiprocessing.Pool(processes=num_processes) as pool:
                pool.map(self._copy_files,file_args)

            print("Data files created successfully.")
        except Exception as e:
            raise ValueError(f"Error creating data files: {e}")


    def transform_to_dictionary(self):
        try:
            file_paths = [self.file_setup,self.file_driver]
            with multiprocessing.Pool() as pool:
                self.dictionaries = pool.map(self.process_file, file_paths)
            return self.dictionaries

        except Exception as e:
            raise ValueError(f"Error converting file '{self.file_setup}' or {self.file_driver} to a dictionary: {e}")
    def target_argument(self,target,target_values):
        
        self.target_string = target
        self.target_values = target_values
        self.target_values = target_values
        for dictionary in self.dictionaries:
            temp_list = []

            if self.target_string in dictionary and dictionary ==  self.dictionaries[0]:
                for value in self.target_values:
                    updated_dict = dictionary.copy()
                    updated_dict[self.target_string] = value
                    self.list_of_modifies_files.append((updated_dict,self.dictionaries[1]))

                    
            if self.target_string in dictionary and dictionary == self.dictionaries[1]:
                 for value in self.target_values:
                    updated_dict = dictionary.copy()
                    updated_dict[self.target_string] = value
                    
                    self.list_of_modifies_files.append((self.dictionaries[0],updated_dict))
        return self.list_of_modifies_files
#     def run_model(self,run_name):
#         program = '/home/kgurbanov/'
#         cmdline = program + ' ' + './launch ' + ' ' + run_name

#         with log_wrapper(cmdline, tag='run') as lw:
#             completed_process = subprocess.run(
#                 cmdline,             # The program + options
#                 shell=True,          # must be used if passing options as str and not list
#                 check=True,          # raise CalledProcessError on failure
#                 #capture_output=True, # collect stdout and stderr; causes memory problems I think
#                 stdout=subprocess.DEVNULL,
#                 stderr=subprocess.DEVNULL,
#                 cwd=rundirectory    # control context
#             )

#             if not completed_process.returncode == 0:
#                 print(completed_process.stdout)
#                 print(completed_process.stderr)
    @staticmethod
    def create_directory(directory):
        os.makedirs(directory, exist_ok=True)
        print(f"Directory created: {directory}")
    def create_project_parallel(self, project_name):
        # Determine the operating system
        if os.name == "posix":
            print("Operating system: Linux or macOS")
            OS = "linux" if os.uname().sysname == "Linux" else "OSX"
        else:
            print("Unknown operating system")
            return

        
        directories = [
            f"results/{project_name}/everystep",
            f"results/{project_name}/netcdf",
            f"results/{project_name}/time_series",
            f"results/{project_name}/hourly",
            f"results/{project_name}/monthly",
            f"results/{project_name}/daily",
        ]

        with multiprocessing.Pool() as pool:
            pool.map(self.create_directory, directories)

        # Modify driver file
        if OS == "linux":
            with open("driver_file.dat", "r") as file:
                lines = file.readlines()
            with open("driver_file.dat", "w") as file:
                for line in lines[1:]:
                    file.write(line)
                file.write(f"setup/{project_name}_driver.dat\n")

        elif OS == "OSX":
            with open("driver_file.dat", "r") as file:
                lines = file.readlines()
            with open("driver_file.dat", "w") as file:
                for line in lines[1:]:
                    file.write(line)
                file.write(f"setup/{project_name}_driver.dat\n")

        # Modify setup file
        if OS == "linux":
            with open("setup_file.dat", "r") as file:
                lines = file.readlines()
            with open("setup_file.dat", "w") as file:
                for line in lines[1:]:
                    file.write(line)
                file.write(f"setup/{project_name}_setup.dat\n")

        elif OS == "OSX":
            with open("setup_file.dat", "r") as file:
                lines = file.readlines()
            with open("setup_file.dat", "w") as file:
                for line in lines[1:]:
                    file.write(line)
                file.write(f"setup/{project_name}_setup.dat\n")

        # Check if necessary files exist
        def check_file(file_path):
            if not os.path.isfile(file_path):
                print(f"Warning: The file {file_path} does not exist")

        check_file(f"./setup/{project_name}_setup.dat")
        check_file(f"./setup/{project_name}_driver.dat")
        check_file(f"./data/{project_name}.dat")

        print("Project for LAKE model created successfully.")
        
        
    #create multiple projects
    def create_multiple_projects(self, project_names):
        processes = []
        for project_name in project_names:
            process = multiprocessing.Process(target=self.create_project_parallel, args=(project_name,))
            process.daemon = False  # Set daemon to False to avoid the AssertionError
            processes.append(process)

        for process in processes:
            process.start()

        for process in processes:
            process.join()

        print("All projects created successfully.")
    def run_experiment(self, experiment_name,rundirectory):
        if experiment_name:
            os.makedirs(f"results/{experiment_name}", exist_ok=True)


        self.run_model(rundirectory)


        if experiment_name:
            basename = experiment_name.rsplit("_", 1)[0]
            source_dir = f"results/{basename}"
            destination_dir = f"results/{experiment_name}"
            self._copy_files(source_dir, destination_dir)

    def run_experiment_parallel(self, experiment_names):
        with multiprocessing.Pool() as pool:
            pool.map(self.run_experiment, experiment_names)

        print("All experiments completed successfully.")
    def run_model(self,rundirectory):
        program_path = "./lake.out"

        completed_process = subprocess.run(
            program_path,
            shell=True,        # Use shell=True if the program is a shell command
            check=True,        # Raise an exception if the process returns non-zero exit code
            cwd=rundirectory   # Set the working directory for the subprocess
        )

        if completed_process.returncode != 0:
            raise RuntimeError(f"Error occurred during model run. Exit code: {completed_process.returncode}")

        print("Model run completed successfully.")


In [9]:
sensitivity = SensitivityAnalysis()
sensitivity.transform_to_dictionary()
sensitivity.target_argument('dataname',["YKD0","YKD1","YKD2"])
folder = ["YKD0-burned","YKD1-burned","YKD2-burned"]
sensitivity.create_files()
rundirectory = os.path.abspath(os.getcwd())
sensitivity.create_data_file()
sensitivity.create_multiple_projects(folder)
sensitivity.run_experiment(folder,rundirectory)


Data files created successfully.
Operating system: Linux or macOS
Operating system: Linux or macOS
Operating system: Linux or macOS
Directory created: results/YKD0-burned/hourly
Directory created: results/YKD2-burned/everystep
Directory created: results/YKD1-burned/everystep
Directory created: results/YKD1-burned/time_series
Directory created: results/YKD1-burned/hourly
Directory created: results/YKD2-burned/time_series
Directory created: results/YKD1-burned/netcdf
Directory created: results/YKD2-burned/hourly
Directory created: results/YKD1-burned/monthly
Directory created: results/YKD1-burned/daily
Directory created: results/YKD0-burned/everystep
Directory created: results/YKD0-burned/netcdf
Directory created: results/YKD0-burned/time_series
Directory created: results/YKD2-burned/monthly
Directory created: results/YKD2-burned/netcdf
Directory created: results/YKD0-burned/monthly
Directory created: results/YKD2-burned/daily
Directory created: results/YKD0-burned/daily
Project for LAKE

CalledProcessError: Command './lake.out' returned non-zero exit status 2.

In [21]:
ls setup/driver_file_0.dat


setup/driver_file_0.dat
